In [2]:
from datasets import load_dataset
import pandas as pd
import torch
import os
from tqdm import tqdm


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
df_all_clues = pd.read_csv("all_clues.csv")
df_all_clues

,CLUE_ID,text,ISO2,clue_type,ISO3
0,0,"In Street View, if the sun is clearly in the N...",[],0,[]
1,1,If the tip of a shadow is pointing south then ...,[],0,[]
2,2,The direction that the shadow’s tip points gen...,[],0,[]
3,3,If the sun is in the Northern Hemisphere then ...,[],0,[]
4,4,On some occasions the sun will appear overhead...,[],0,[]
...,...,...,...,...,...
4130,4130,Subtropical with rapid changes in temperature ...,['PY'],1,['PRY']
4131,4131,The weather in Peru varies according to area –...,['PE'],1,['PER']
4132,4132,"Tropical and humid, cooled by the northeast tr...",['SR'],1,['SUR']
4133,4133,Uruguay has an exceptionally fine temperate cl...,['UY'],1,['URY']


In [6]:
df_one_hot_enc = pd.read_csv("encodings.csv")
df_one_hot_enc

,ISO2,encoding
0,AD,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,AE,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,AF,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,AG,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,AI,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
216,XK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
217,YE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
218,ZA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
219,ZM,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
from datasets import Dataset

def encode_dataset(df, tokenizer, model):
    encodings = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Encoding rows"):
        clue_id = row[0]
        text = row[1]
        iso_2 = row[2]
        clue_type = row[3]
        iso_3 = row[4]

        # Tokenize input and propagate through model
        inputs = tokenizer(discription, return_tensors='pt', truncation=True).to('cuda')
        with torch.no_grad():
            outputs = model(**inputs)

        # The outputs include the last hidden state, pooler output, and hidden states
        # We are interested in the last hidden state
        encoding = outputs.last_hidden_state.cpu().numpy()  # Convert tensor to numpy
        encodings.append({"clue_id": identifier, "enc": encoding, "iso2" : iso2, "clue_type": clue_type, "iso3" : iso3} )


    return Dataset.from_list(encodings)

In [ ]:
#enc_data_dict.push_to_hub("gips-mai/clues_enc")

dataset = encode_dataset(df_all_clues, tokenizer, model)
